In [1]:
import websocket
import pytz
import hmac
import hashlib
import base64
import json
import requests
import pandas as pd

from datetime import datetime
from dotenv import dotenv_values
from collections import defaultdict

In [2]:
keys = dotenv_values(".env")

endpoint = "/api/v3/brokerage/products"
url = "https://coinbase.com" + endpoint

timestamp = int(datetime.now().timestamp())
body = ""
message = str(timestamp) + "GET" + endpoint + body
signature = hmac.new(keys["COINBASE_SECRET"].encode('utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).digest()
headers = {"accept": "application/json",
           "CB-ACCESS-KEY": keys["COINBASE_PUBLIC"],
           "CB-ACCESS-SIGN": signature.hex(),
           "CB-ACCESS-TIMESTAMP": str(timestamp)}

payload = {"product_type" : "SPOT"}

r = requests.get(url, params=payload, headers=headers)

In [3]:
usds = ["USD", "USDC", "USDT", "EUR", "GBP"]
non_usds = []
btc_pairs = []
trage_pairs = ["BTC-USD"]
for prod in r.json()["products"]:
    pid = prod["product_id"]
    pid0, pid1 = pid.strip().split("-")

    if pid0 not in usds and pid1 not in usds:
        non_usds.append(pid)
    if pid1 == "BTC":
        btc_pairs.append(pid)
        trage_pairs.append(pid)
        trage_pairs.append(f"{pid0}-USD")

non_usds.sort()
#non_usds
trage_pairs

['BTC-USD',
 'ETH-BTC',
 'ETH-USD',
 'SOL-BTC',
 'SOL-USD',
 'WBTC-BTC',
 'WBTC-USD',
 'AVAX-BTC',
 'AVAX-USD',
 'FIL-BTC',
 'FIL-USD',
 'DOT-BTC',
 'DOT-USD',
 'BCH-BTC',
 'BCH-USD',
 'LTC-BTC',
 'LTC-USD',
 'LINK-BTC',
 'LINK-USD',
 'GRT-BTC',
 'GRT-USD',
 'ADA-BTC',
 'ADA-USD',
 'ICP-BTC',
 'ICP-USD',
 'DOGE-BTC',
 'DOGE-USD',
 'MATIC-BTC',
 'MATIC-USD',
 'XLM-BTC',
 'XLM-USD',
 'ATOM-BTC',
 'ATOM-USD',
 'XTZ-BTC',
 'XTZ-USD',
 'AAVE-BTC',
 'AAVE-USD',
 'UNI-BTC',
 'UNI-USD',
 'MANA-BTC',
 'MANA-USD',
 'ALGO-BTC',
 'ALGO-USD',
 '1INCH-BTC',
 '1INCH-USD',
 'CGLD-BTC',
 'CGLD-USD',
 'SNX-BTC',
 'SNX-USD',
 'YFI-BTC',
 'YFI-USD',
 'MKR-BTC',
 'MKR-USD',
 'LRC-BTC',
 'LRC-USD',
 'ETC-BTC',
 'ETC-USD',
 'EOS-BTC',
 'EOS-USD',
 'COMP-BTC',
 'COMP-USD',
 'BAT-BTC',
 'BAT-USD',
 'ENJ-BTC',
 'ENJ-USD',
 'CRV-BTC',
 'CRV-USD',
 'DASH-BTC',
 'DASH-USD',
 'ZEC-BTC',
 'ZEC-USD',
 'BAL-BTC',
 'BAL-USD',
 'ANKR-BTC',
 'ANKR-USD',
 'AXS-BTC',
 'AXS-USD']

In [4]:
keys = dotenv_values(".env")

endpoint = "/api/v3/brokerage/products"
url = "https://coinbase.com" + endpoint

timestamp = int(datetime.now().timestamp())
body = ""
message = str(timestamp) + "GET" + endpoint + body
signature = hmac.new(keys["COINBASE_SECRET"].encode('utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).digest()

headers = {"accept": "application/json",
           "CB-ACCESS-KEY": keys["COINBASE_PUBLIC"],
           "CB-ACCESS-SIGN": signature.hex(),
           "CB-ACCESS-TIMESTAMP": str(timestamp)}

#payload = {"product_type": "SPOT", "product_ids": trage_pairs[:5]}
payload = {"product_type": "SPOT", "product_ids": trage_pairs} 

r = requests.get(url, params=payload, headers=headers)

In [5]:
pricemap = {}
for prod in r.json()["products"]:
    pricemap[prod["product_id"]] = float(prod["price"])

    splitprod = prod["product_id"].split("-") 
    revprod = splitprod[1] + "-" + splitprod[0]
    pricemap[revprod] = 1.0/float(prod["price"])

print(json.dumps(pricemap, indent=2))

{
  "BTC-USD": 45240.87,
  "USD-BTC": 2.2103907374018226e-05,
  "ETH-USD": 2365.422,
  "USD-ETH": 0.0004227575460108175,
  "SOL-USD": 109.23,
  "USD-SOL": 0.009154994049253868,
  "AVAX-USD": 41.23,
  "USD-AVAX": 0.02425418384671356,
  "LINK-USD": 15.389,
  "USD-LINK": 0.06498148027812074,
  "ICP-USD": 13.938,
  "USD-ICP": 0.07174630506528913,
  "FIL-USD": 7.357,
  "USD-FIL": 0.13592496941688187,
  "MATIC-USD": 0.987,
  "USD-MATIC": 1.0131712259371835,
  "DOGE-USD": 0.09159,
  "USD-DOGE": 10.918222513374822,
  "ADA-USD": 0.6159,
  "USD-ADA": 1.6236402013313849,
  "GRT-USD": 0.2091,
  "USD-GRT": 4.782400765184122,
  "DOT-USD": 8.476,
  "USD-DOT": 0.11798017932987256,
  "LTC-USD": 73.43,
  "USD-LTC": 0.013618412093149937,
  "BCH-USD": 258.18,
  "USD-BCH": 0.0038732667131458672,
  "AAVE-USD": 112.37,
  "USD-AAVE": 0.00889917237696894,
  "XLM-USD": 0.130384,
  "USD-XLM": 7.669652718124923,
  "ATOM-USD": 11.153,
  "USD-ATOM": 0.08966197435667533,
  "UNI-USD": 7.337,
  "USD-UNI": 0.1362954886

In [6]:
TAKER_FEE = 0.0045
MAKER_FEE = 0.0025

def get_fee_tier(volume):
    if volume >= 50000:
        return 0.0025, 0.0015 
    elif volume >= 10000:
        return 0.0040, 0.0025 
    elif volume >= 1000:
        return 0.0055, 0.0035
    else:
        return 0.0080, 0.0060

In [7]:
def compute_trade_result(inv_in, triplet):
    fees = 0
    inv_out = inv_in
    c0, c1, c2 = triplet.strip().split("-")

    for p in [[f"{c0}-{c1}", MAKER_FEE], [f"{c1}-{c2}", MAKER_FEE], [f"{c2}-{c0}", MAKER_FEE]]:
        fee = inv_out * p[1]
        inv_out *= pricemap[p[0]]

        #convert fee to usd and add to total
        pid, _ = p[0].strip().split("-")
        if pid != "USD":
            fees += fee * pricemap[f"{pid}-USD"]
        else:
            fees += fee

    subtotal = inv_out
    total = subtotal - fees
    profit = total - inv_in

    return subtotal, total, profit, fees


In [9]:
inv_in = 1000.0
TAKER_FEE, MAKER_FEE = get_fee_tier(inv_in)

data = defaultdict(list)

for pair in btc_pairs:
    pid0, _ = pair.strip().split("-")
    #test both direction of each triplet
    for triplet in [f"USD-BTC-{pid0}", f"USD-{pid0}-BTC"]:
        subtotal, total, profit, fees = compute_trade_result(inv_in, triplet)

        data["triplet"].append(triplet)
        data["total"].append(total)
        data["subtotal"].append(subtotal)
        data["profit"].append(profit)
        data["fees"].append(fees)


pd.set_option('display.max_rows', 200)
df = pd.DataFrame(
    data={
        "Triplet": data["triplet"],
        "Subtotal": data["subtotal"],
        "Total": data["total"],
        "Fees": data["fees"],
        "Profit": data["profit"],
        }).sort_values(by="Profit", ascending=False)

df

,Triplet,Subtotal,Total,Fees,Profit
64,USD-BTC-CRV,1009.251597,998.719216,10.532381,-1.280784
72,USD-BTC-ANKR,1007.938176,997.410393,10.527784,-2.589607
66,USD-BTC-DASH,1007.158755,996.633699,10.525056,-3.366301
68,USD-BTC-ZEC,1006.020553,995.499481,10.521072,-4.500519
44,USD-BTC-CGLD,1005.617143,995.097483,10.519660,-4.902517
75,USD-AXS-BTC,1004.561050,994.045087,10.515964,-5.954913
26,USD-BTC-MATIC,1004.445515,993.929955,10.515559,-6.070045
5,USD-WBTC-BTC,1003.657600,993.144799,10.512802,-6.855201
22,USD-BTC-ICP,1003.205018,992.693800,10.511218,-7.306200
36,USD-BTC-UNI,1002.946001,992.435690,10.510311,-7.564310
